<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelsDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [35]:
# Importiamo il dataset ottenuto tramite le operazioni di Data Preprocessing
dataset = pd.read_csv("tennis.csv")

#**Train-Test split**

In [36]:
# Estrai tutti gli ID unici delle partite
match_ids = dataset['match_id'].unique()

# Split degli ID in training e test (es. 80% train, 20% test)
train_ids, test_ids = train_test_split(match_ids, test_size=0.2, random_state=42)

# Filtra il dataset mantenendo intere le partite
train_data = dataset[dataset['match_id'].isin(train_ids)].reset_index(drop=True)
test_data = dataset[dataset['match_id'].isin(test_ids)].reset_index(drop=True)

Adesso aumento la dimensionalità degli split considerando le istanze simmetriche

In [37]:
def augment_with_symmetric(df: pd.DataFrame) -> pd.DataFrame:
    df_swapped = df.copy()

    # Colonne da swappare a coppie
    swap_pairs = [
        ('Set1', 'Set2'),
        ('Gm1', 'Gm2'),
        ('Player1', 'Player2'),
        ('Ranking1', 'Ranking2'),
        ('p1_win_nobreak_point', 'p2_win_nobreak_point'),
        ('p1_win_break_point', 'p2_win_break_point'),
        ('p1_lost_nobreak_point', 'p2_lost_nobreak_point'),
        ('p1_lost_break_point', 'p2_lost_break_point'),
        ('serve_ace_1', 'serve_ace_2'),
        ('serve_miss2_1', 'serve_miss1_2'),
        ('rally_winner_1', 'rally_winner_2'),
        ('rally_forced2_1', 'rally_forced1_2'),
        ('rally_unforced1_2', 'rally_unforced2_1')
    ]
    for col1, col2 in swap_pairs:
        df_swapped[[col1, col2]] = df[[col2, col1]].values

    # Colonne dove 1 <-> 2
    invert_1_2_cols = ['Svr', 'PtWinner', 'SetWinner', 'MatchWinner']
    for col in invert_1_2_cols:
        df_swapped[col] = df[col].replace({1: 2, 2: 1})

    # Inversione della colonna Pts (es. "40-15" -> "15-40")
    def invert_pts(value):
        try:
            left, right = value.split('-')
            return f"{right}-{left}"
        except:
            return value  # Se non è nel formato previsto, lo lascia invariato

    if 'Pts' in df.columns:
        df_swapped['Pts'] = df['Pts'].apply(invert_pts)

    return pd.concat([df, df_swapped], ignore_index=True)


train_data = augment_with_symmetric(train_data)
test_data = augment_with_symmetric(test_data)

Bisogna convertire i valori non numerici in valori numerici e poi bisogna convertire i valori numerici in valori adatti all'addestramento

Creare i batch di training del Modello 1(il migliore su Github): Quindi sono con i ranking e chi vince

Usare la predizione di partenza di tale modello per:

Modello 2 LSTM: per ogni match id considerare tutte le istanze di punteggi con le colonne utili con etichetta match winner
Provare variante sui set quindi dividere per set id e mettere etichetta SetWinner

Crea la Loss Function corretta dando peso alle giuste feature e ai punti consecutivi


Considera di usare la predizione del modello 1 con la percentuale data dalla funzione matematica

#**Model 1**

# **Model 2**